In [1]:
import os
import pandas as pd
import json
import folium

In [2]:
##### Manually changed items #####
code = '03d-03'
title = 'Original Land Survey Maps: Iowa'

In [3]:
df = pd.read_csv(os.path.join('data', code, code+'.csv'))[['Title', 'Bounding Box', 'Information', 'Identifier']]
df.head()

,Title,Bounding Box,Information,Identifier
0,"Land Survey Map: Des Moines County, Iowa; 69N 3W","-91.25805435, 40.72396221, -91.14168765, 40.81...",https://digital.lib.uiowa.edu/islandora/object...,a7477991-2669-43d9-b0b2-3968ad7e309c
1,"Land Survey Map: Des Moines County, Iowa; Lee ...","-91.37260569, 40.72501917, -91.25684888, 40.81...",https://digital.lib.uiowa.edu/islandora/object...,37c2e51e-d5d9-4ca0-aa5f-71a26a62981e
2,"Land Survey Map: Des Moines County, Iowa; 70N 1W","-91.02658781, 40.88003418, -91.01171581, 40.89...",https://digital.lib.uiowa.edu/islandora/object...,53438ec7-b886-4afe-b362-cca9ee094b3d
3,"Land Survey Map: Des Moines County, Iowa; 70N 3W","-91.25684888, 40.8113462, -91.14135497, 40.900...",https://digital.lib.uiowa.edu/islandora/object...,8bfcb12d-ec8e-46c7-8a81-5f1981c189f6
4,"Land Survey Map: Henry County, Iowa; 70N 5W","-91.48750768, 40.81257309, -91.37153746, 40.90...",https://digital.lib.uiowa.edu/islandora/object...,09c44cf8-8afc-4b2f-8e8c-8cd6e2c05bc1


In [4]:
df = pd.concat([df, df['Bounding Box'].str.split(',', expand=True).astype(float)], axis=1).rename(
    columns={0:'minX', 1:'minY', 2:'maxX', 3:'maxY'})
df['maxXmaxY'] = df.apply(lambda row: [row.maxX, row.maxY], axis = 1)
df['maxXminY'] = df.apply(lambda row: [row.maxX, row.minY], axis = 1)
df['minXminY'] = df.apply(lambda row: [row.minX, row.minY], axis = 1)
df['minXmaxY'] = df.apply(lambda row: [row.minX, row.maxY], axis = 1)
df_clean = df.drop(columns =['minX', 'minY', 'maxX', 'maxY'])
df_clean.head()

,Title,Bounding Box,Information,Identifier,maxXmaxY,maxXminY,minXminY,minXmaxY
0,"Land Survey Map: Des Moines County, Iowa; 69N 3W","-91.25805435, 40.72396221, -91.14168765, 40.81...",https://digital.lib.uiowa.edu/islandora/object...,a7477991-2669-43d9-b0b2-3968ad7e309c,"[-91.14168765, 40.81259418]","[-91.14168765, 40.72396221]","[-91.25805435, 40.72396221]","[-91.25805435, 40.81259418]"
1,"Land Survey Map: Des Moines County, Iowa; Lee ...","-91.37260569, 40.72501917, -91.25684888, 40.81...",https://digital.lib.uiowa.edu/islandora/object...,37c2e51e-d5d9-4ca0-aa5f-71a26a62981e,"[-91.25684888, 40.81259418]","[-91.25684888, 40.72501917]","[-91.37260569, 40.72501917]","[-91.37260569, 40.81259418]"
2,"Land Survey Map: Des Moines County, Iowa; 70N 1W","-91.02658781, 40.88003418, -91.01171581, 40.89...",https://digital.lib.uiowa.edu/islandora/object...,53438ec7-b886-4afe-b362-cca9ee094b3d,"[-91.01171581, 40.89871875]","[-91.01171581, 40.88003418]","[-91.02658781, 40.88003418]","[-91.02658781, 40.89871875]"
3,"Land Survey Map: Des Moines County, Iowa; 70N 3W","-91.25684888, 40.8113462, -91.14135497, 40.900...",https://digital.lib.uiowa.edu/islandora/object...,8bfcb12d-ec8e-46c7-8a81-5f1981c189f6,"[-91.14135497, 40.90001049]","[-91.14135497, 40.8113462]","[-91.25684888, 40.8113462]","[-91.25684888, 40.90001049]"
4,"Land Survey Map: Henry County, Iowa; 70N 5W","-91.48750768, 40.81257309, -91.37153746, 40.90...",https://digital.lib.uiowa.edu/islandora/object...,09c44cf8-8afc-4b2f-8e8c-8cd6e2c05bc1,"[-91.37153746, 40.90057979]","[-91.37153746, 40.81257309]","[-91.48750768, 40.81257309]","[-91.48750768, 40.90057979]"


In [5]:
# create_geojson_features 
def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'title': title,
        'features': features
    }
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'id': row['Identifier'],
            'geometry': {
                'type':'Polygon', 
                'coordinates':[[row['maxXmaxY'], row['maxXminY'], row['minXminY'], row['minXmaxY'], row['maxXmaxY']], ]
            },
            'properties': {
                'label': row['Title'],
                'Name': row['Title'],
                'recordIdentifier': row['Identifier'],
                'websiteUrl': row['Information']
            }
           }

        features.append(feature)
    return geojson

data_geojson = create_geojson_features(df_clean)

> Creating GeoJSON features...


In [6]:
with open(os.path.join('data', code, code+'.geojson'), 'w') as txtfile:
    json.dump(data_geojson, txtfile)
print('> Creating GeoJSON file...')

> Creating GeoJSON file...


In [7]:
print('> Making map...')
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 7)
folium.GeoJson(open(os.path.join('data', code, code+'.geojson'), 'r').read(),
               tooltip = folium.GeoJsonTooltip(fields=('Name', 'websiteUrl'),
                                               aliases=('Name','websiteUrl')),
               show = True).add_to(m)
m

> Making map...
